# 数据前处理以及构建基因显著性矩阵
## 20231104
## Dataset: Lung
### 1, Down sampling from single-cell RNA-seq data for construction of Significant Matrix
### 2, Create pseudo-bulk and simulate different tissue states

In [2]:
import pandas as pd
import numpy as np
import anndata
import scanpy as sc
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib as mat
import os
import sys
from scipy import stats
import warnings
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import random
from  tqdm import tqdm


warnings.filterwarnings("ignore")

# 1, 构建测试Lung 显著性矩阵

In [17]:
# 下采样，构建显著性表达矩阵

sc_adata = sc.read_h5ad("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/raw/Lung_counts.h5ad")

N = 20
sc_adata_20 = sc_adata[sc_adata.obs.groupby("CellType").sample(n=N,random_state=1,replace=False).index].copy()

sc_adata_20.obs.celltype = sc_adata_20.obs.CellType

sc_adata_20.obs.celltype.to_csv("01_datasets/raw/Lung_celltype9.csv")

sc_adata_20.to_df().T.to_csv("01_datasets/raw/Lung_counts_celltype9_down20.csv")

sc_adata_20.write_h5ad("01_datasets/raw/Lung_counts_down20_celltype9.h5ad")
sc_adata_20.obs.celltype.to_csv("01_datasets/raw/Lung_celltype9.txt",sep="\t")

sc_adata_20.to_df().T.to_csv("01_datasets/raw/Lung_counts_celltype9_down20.txt",sep="\t")
# input for CIBERSROTx


In [60]:
# 基于CIBERSORTx的结果，构建single cell h5ad
sig_data = pd.read_table("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/data/Lung_celltype9_sig_matrix.txt",sep="\t",index_col=0)

sig_adata = sc.AnnData(X=csr_matrix(sig_data.T))
sig_adata.var_names = sig_data.T.columns.values
sig_adata.obs_names = sig_data.columns.values
sig_adata.obs["celltype"] = sig_data.columns.values

sig_adata.obs.celltype = sig_adata.obs.celltype.astype('category')

sig_adata.write_h5ad("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/data/Lung_celltype9_sig_counts.h5ad")

In [46]:
# 构建pseudo bulk h5ad files from simulted data
# create Bulk Protein h5ad files
bulk_data = pd.read_table("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/raw/Lung_pseudobulk_counts.txt",sep="\t",index_col=0)
meta_data = pd.read_table("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/raw/Lung_pseudobulk_counts_label.txt",sep="\t",index_col=0)
bulk_adata = sc.AnnData(X=csr_matrix(bulk_data.T))
bulk_adata.var_names = bulk_data.T.columns.values
bulk_adata.obs_names = bulk_data.columns.values

bulk_adata.uns["celltype_gd"] = meta_data

bulk_adata.write_h5ad("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/data/Lung_pseudobulk_counts.h5ad")

In [27]:
# 2, 构建Pseudo bulk

In [3]:

def generate_simulated_data(prop,sc_data, cell_type_obs="celltype"):

    if isinstance(sc_data.X, np.ndarray):
        pass
    else:
        sc_data.X = sc_data.X.toarray() #73 * 10466

    sc_data = pd.DataFrame(sc_data.X, index=sc_data.obs[cell_type_obs], columns=sc_data.var.index)
    # sc_data.dropna(inplace=True)
    sc_data[cell_type_obs] = sc_data.index
    sc_data.index = range(len(sc_data))
    celltype_groups = sc_data.groupby(cell_type_obs).groups

    sc_data.drop(columns=cell_type_obs, inplace=True)

    sc_data = anndata.AnnData(sc_data)
    sc_data = sc_data.X
    sc_data = np.ascontiguousarray(sc_data, dtype=np.float32)

    for key, value in celltype_groups.items():
        celltype_groups[key] = np.array(value)    

    n = np.random.randint(100,20000,size=prop.shape[0]).reshape(-1,1) # 模拟bulk中细胞的整体数量
    cell_num = np.floor(n * prop)

    # precise proportion based on cell_num
    prop = cell_num / np.sum(cell_num, axis=1).reshape(-1, 1)
    sample = np.zeros((prop.shape[0], sc_data.shape[1])) # 1000 * 253
    
    allcellname = celltype_groups.keys() # dict_keys, 19

    for i, sample_prop in tqdm(enumerate(cell_num)): # 每个pseudo-spot中对应的cell-type的细胞数量
        for j, cellname in enumerate(allcellname): # 对应的cell-type[1-19]
            select_index = np.random.choice(celltype_groups[cellname], size=int(sample_prop[j]), replace=True) # 从cell-type抽取该数量的cell
            sample[i] += sc_data[select_index].sum(axis=0) # 对于同种类型细胞，进行累加。输出单个spot，单个细胞类型对应的基因表达量。


    _spot_data = sample
    _spot_type = prop
    return (_spot_data,_spot_type)

In [26]:
# create pseudo bulk h5ad files with all genes.
# test different conditions: Rare, Normal, Similar
sc_adata = sc.read_h5ad("/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/raw/Lung_counts.h5ad")
celltype_order = ['B cell', 'T cell',
       'ciliated columnar cell of tracheobronchial tree',
       'classical monocyte', 'leukocyte', 'lung endothelial cell',
       'myeloid cell', 'natural killer cell', 'stromal cell']
for state in ["Rare", "Normal", "Similar"]:
       if (state == "Rare"):
              prop = np.random.dirichlet(np.ones(9)*0.5, 100)
       elif (state == "Normal"):
              prop = np.random.dirichlet(np.ones(9)*1, 100)
       elif (state == "Similar"):
              prop = np.random.dirichlet(np.ones(9)*2, 100)

       print(f"==== {state}======")

       for seed in [0]:
              random.seed(seed)

              spot_data,temp_prop = generate_simulated_data(prop=prop, sc_data=sc_adata,cell_type_obs="celltype")

              sp_adata = sc.AnnData(X=csr_matrix(spot_data))
              sp_adata.var_names  = sc_adata.var_names

              sp_adata.uns['celltype_gd'] = pd.DataFrame(temp_prop,columns= celltype_order,
                                                        index=sp_adata.obs.index)
              
              sp_adata.write_h5ad(f"/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/simulated/Lung_sim_{state}_{seed}.h5ad")
              sp_adata.uns['celltype_gd'].to_csv(f"/aaa/zihanwu/yyyli2/projectx_General_Deconv/01_datasets/simulated/Lung_sim_{state}_{seed}.csv")

==== Rare======


100it [01:18,  1.28it/s]


==== Normal======


100it [01:33,  1.06it/s]


==== Similar======


100it [01:17,  1.29it/s]
